In [1]:
#Import the dependencies.
import pandas as pd
import gmaps
import requests

#Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Thompson,CA,2021-04-13 06:17:35,55.7435,-97.8558,21.20,53,90,21.85
1,1,Punta Arenas,CL,2021-04-13 06:17:35,-53.1500,-70.9167,42.80,75,40,20.71
2,2,Hasaki,JP,2021-04-13 06:17:35,35.7333,140.8333,66.00,68,75,11.50
3,3,Geraldton,AU,2021-04-13 06:17:36,-28.7667,114.6000,71.60,49,40,17.27
4,4,Itoman,JP,2021-04-13 06:17:36,26.1247,127.6694,78.01,83,75,9.22


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [13]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [14]:
fig = gmaps.figure(center = (30.0,31), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
#Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [20]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & \
                                       (city_data_df['Max Temp']>= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Itoman,JP,2021-04-13 06:17:36,26.1247,127.6694,78.01,83,75,9.22
6,6,Sao Filipe,CV,2021-04-13 06:17:37,14.8961,-24.4956,76.15,63,77,8.97
12,12,Manokwari,ID,2021-04-13 06:17:38,-0.8667,134.0833,82.26,79,37,3.33
13,13,Ternate,ID,2021-04-13 06:17:39,0.8000,127.4000,84.11,70,71,9.01
17,17,Bosaso,SO,2021-04-13 06:17:40,11.2842,49.1816,82.40,65,0,25.32
18,18,Maragogi,BR,2021-04-13 06:17:40,-9.0122,-35.2225,77.81,85,93,3.69
19,19,Puerto Ayora,EC,2021-04-13 06:17:41,-0.7393,-90.3518,75.00,98,100,3.47
22,22,Abonnema,NG,2021-04-13 06:17:42,4.7084,6.7931,79.23,83,77,4.52
27,27,Rikitea,PF,2021-04-13 06:13:25,-23.1203,-134.9692,78.33,69,4,9.24
32,32,Alofi,NU,2021-04-13 06:17:46,-19.0595,-169.9187,75.20,94,75,1.14


In [21]:
preferred_cities_df.count()

City_ID       150
City          150
Country       146
Date          150
Lat           150
Lng           150
Max Temp      150
Humidity      150
Cloudiness    150
Wind Speed    150
dtype: int64

In [22]:
#Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Itoman,JP,78.01,26.1247,127.6694,
6,Sao Filipe,CV,76.15,14.8961,-24.4956,
12,Manokwari,ID,82.26,-0.8667,134.0833,
13,Ternate,ID,84.11,0.8000,127.4000,
17,Bosaso,SO,82.40,11.2842,49.1816,
18,Maragogi,BR,77.81,-9.0122,-35.2225,
19,Puerto Ayora,EC,75.00,-0.7393,-90.3518,
22,Abonnema,NG,79.23,4.7084,6.7931,
27,Rikitea,PF,78.33,-23.1203,-134.9692,
32,Alofi,NU,75.20,-19.0595,-169.9187,


In [34]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [43]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Itoman,JP,78.01,26.1247,127.6694,Southern Beach Hotel & Resort Okinawa
6,Sao Filipe,CV,76.15,14.8961,-24.4956,Tortuga B&B
12,Manokwari,ID,82.26,-0.8667,134.0833,Swiss Belhotel Manokwari
13,Ternate,ID,84.11,0.8000,127.4000,Grand Dafam Bela Ternate
17,Bosaso,SO,82.40,11.2842,49.1816,Red Sea Hotel


In [44]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [48]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [49]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))